In [35]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [36]:
browser = Chrome('/Users/ajsaule/Desktop/chromedriver')

In [37]:
url = 'https://stat.gov.kz/official/industry/26/statistic/6'
browser.get(url)

In [38]:
lang = browser.find_element_by_xpath('//*[@id="header"]/div/div/div[3]/div[2]/ul/li[2]/button')
lang.click()

In [39]:
period = browser.find_element_by_xpath('//*[@id="selPeriodities"]')
period.click()
c_p = Select(period)
c_p.select_by_visible_text('Неделя')

In [40]:
named = browser.find_element_by_xpath('//*[@id="selNames"]')
named.click()
c_n = Select(named)
c_n.select_by_visible_text('Индекс цен и средние цены на социально-значимые продовольственные товары в Республике Казахстан')

In [1]:
years=['2022']
for year in range(len(years)):
    #sleep(8)
    p_years = browser.find_element_by_xpath('//*[@id="selPeriods"]')
    p_years.click()
    c_y = Select(p_years)
    c_y.select_by_visible_text(years[year])

NameError: name 'sleep' is not defined

In [2]:
import time

In [92]:
for idx,file in enumerate(browser.find_elements_by_xpath('.//div[@class="trud-archive-list"]/a')):
    if 0<=idx<=8:
        file.click()
        time.sleep(5)
        url = file.get_attribute('href')  
        filename = urllib.request.urlopen(url).info().get_filename()
        filename = 'Э'+filename[1:8]+'Н'+filename[9:34]+'рус'+filename[37:]
        print(filename)
        src = "/Users/ajsaule/Downloads/"+filename
        dest = "/Users/ajsaule/Desktop/2022/5 jan - 1 mar/"+filename
        os.rename(src, dest)
    elif 9<=idx<=10:
        file.click()
        time.sleep(5)
        url = file.get_attribute('href')  
        filename = urllib.request.urlopen(url).info().get_filename()
        filename = 'Э'+filename[1:8]+'Н'+filename[9:34]+'рус'+filename[37:]
        print(filename)
        src = "/Users/ajsaule/Downloads/"+filename
        dest = "/Users/ajsaule/Desktop/2022/9-15 mar/"+filename
        os.rename(src, dest)
    else:
        file.click()
        time.sleep(5)
        url = file.get_attribute('href')  
        filename = urllib.request.urlopen(url).info().get_filename()
        filename = 'Э'+filename[1:8]+'Н'+filename[9:34]+'рус'+filename[37:]
        print(filename)
        src = "/Users/ajsaule/Downloads/"+filename
        dest = "/Users/ajsaule/Desktop/2022/24 mar-17 may/"+filename
        os.rename(src, dest)

Э-18-20-Н (28.12.2021-05.01.2022) рус.docx
Э-18-14-Н (05.01.2022-11.01.2022) рус.docx
Э-18-14-Н (11.01.2022-18.01.2022) рус.docx
Э-18-14-Н (18.01.2022-25.01.2022) рус.docx
Э-18-20-Н (25.01.2022-01.02.2022) рус.docx
Э-18-20-Н (01.02.2022-08.02.2022) рус.docx
Э-18-20-Н (08.02.2022-15.02.2022) рус.docx
Э-18-20-Н (15.02.2022-22.02.2022) рус.docx
Э-18-20-Н (22 02 2022-01 03 2022) рус.docx
Э-18-20-Н (01 03 2022-09 03 2022) рус.docx
Э-18-20-Н (15 03 2022-09 03 2022) рус.docx
Э-18-20-Н (24 03 2022-15 03 2022) рус.docx
Э-18-20-Н (29 03 2022-24 03 2022) рус.docx
Э-18-20-Н (05 04 2022-29 03 2022) рус.docx
Э-18-20-Н (12 04 2022-05 04 2022) рус.docx
Э-18-20-Н (12 04 2022-19 04 2022) рус.docx
Э-18-20-Н (19 04 2022-27 04 2022) рус.docx
Э-18-20-Н (26.04.2022-03.05.2022) рус.docx
Э-18-20-Н (03.05.2022-11.05.2022) рус.docx
Э-18-20-Н (11.05.2022-17.05.2022) рус.docx


In [1]:
import pandas as pd
from docx import Document
import os
import time
import re

In [2]:
dfs=[]

In [3]:
def table_read(document):
    document = Document("/Users/ajsaule/Desktop/2022/5 jan - 1 mar/"+file)
    f=re.findall(r'по.*года', document.paragraphs[1].text)
    for i in range(len(document.tables)):
        if 2<=i<=4:
            table=document.tables[i]
            data = [[cell.text for cell in row.cells] for row in table.rows]
            df = pd.DataFrame(data)
            df['date']=f[0]
            dfs.append(pd.DataFrame(df))

In [4]:
path ='/Users/ajsaule/Desktop/2022/5 jan - 1 mar/'
files = os.listdir(path)
for file in files:
    time.sleep(3)
    if file.endswith('.docx'):
        table_read(file)

In [5]:
dfs_temp=[]
dfs_temp_1=[]
dfs_temp_2=[]

In [6]:
table_1=dfs[0::3]
for t1 in table_1:
    time.sleep(2)
    df_1=pd.DataFrame(t1)
    outside_col, inside_col = df_1.iloc[0], df_1.iloc[1]
    df_1.columns = pd.MultiIndex.from_tuples(list(zip(outside_col,inside_col)))
    df_1=df_1.drop(df_1.index[[0,1]]).reset_index(drop=True).drop('с начала месяца', axis=1, level=1).droplevel(level=1, axis=1).rename( columns={'':'region'})
    df_1=df_1.rename(columns={df_1.columns[7]: 'date'})
    df_1=df_1.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    #df_1 = pd.concat()
    dfs_temp.append(df_1)

dfs_temp = pd.concat(dfs_temp).reset_index().drop(columns=['index'])
time.sleep(2)
table_2=dfs[1::3]
for t2 in table_2:
    time.sleep(3)
    df_2=pd.DataFrame(t2)
    df_2= df_2.rename(columns=df_2.iloc[0]).drop(df_2.index[0]).reset_index(drop=True).rename( columns={'':'region'})
    df_2=df_2.rename(columns={df_2.columns[8]: 'date'})
    df_2=df_2.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    #df_1 = pd.concat()
    dfs_temp_1.append(df_2)

dfs_temp_1 = pd.concat(dfs_temp_1).reset_index().drop(columns=['index'])
time.sleep(2)
table_3=dfs[2::3]
for t3 in table_3:
    time.sleep(3)
    df_3=pd.DataFrame(t3)
    df_3= df_3.rename(columns=df_3.iloc[0]).drop(df_3.index[0]).reset_index(drop=True).rename( columns={'':'region'})
    df_3=df_3.rename(columns={df_3.columns[8]: 'date'})
    df_3=df_3.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    #df_1 = pd.concat()
    dfs_temp_2.append(df_3)

dfs_temp_2 = pd.concat(dfs_temp_2).reset_index().drop(columns=['index'])
time.sleep(2)

In [ ]:
l_pd_2=[dfs_temp, dfs_temp_1, dfs_temp_2]
n_df=pd.concat(l_pd_2)

In [7]:
n_df

,region,date,type_of_goods,price_in_per
0,Нур-Султан,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,2"
1,Алматы,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,1"
2,Шымкент,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,2"
3,Актау,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,1"
4,Актобе,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,4"
...,...,...,...,...
3595,Семей,по 8 февраля 2022 года,Соль,"103,6"
3596,Талдыкорган,по 8 февраля 2022 года,Соль,"100,0"
3597,Тараз,по 8 февраля 2022 года,Соль,"100,0"
3598,Туркестан,по 8 февраля 2022 года,Соль,"100,0"


In [8]:
dfs_1=[]

In [9]:
def table_read_1(document):
    document = Document("/Users/ajsaule/Desktop/2022/9-15 mar/"+file)
    f=re.findall(r'по.*года', document.paragraphs[1].text)
    for i in range(len(document.tables)):
        if 8<=i<=10:
            table=document.tables[i]
            data = [[cell.text for cell in row.cells] for row in table.rows]
            df = pd.DataFrame(data)
            df['date']=f[0]
            dfs_1.append(pd.DataFrame(df))

In [10]:
path ='/Users/ajsaule/Desktop/2022/9-15 mar/'
files = os.listdir(path)
for file in files:
    #time.sleep(3)
    if file.endswith('.docx'):
        table_read_1(file)

In [11]:
dfs_temp_3=[]
dfs_temp_4=[]
dfs_temp_5=[]

In [12]:
table_1=dfs_1[0::3]
for idx,t1 in enumerate(table_1):
    df_1=pd.DataFrame(t1)
    if idx==0:
        df_1= df_1.rename(columns=df_1.iloc[0]).drop(df_1.index[0]).reset_index(drop=True).rename( columns={'':'region'})
        df_1=df_1.rename(columns={df_1.columns[7]: 'date'})
        df_1=df_1.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    elif idx==1:
        outside_col, inside_col = df_1.iloc[0], df_1.iloc[1]
        df_1.columns = pd.MultiIndex.from_tuples(list(zip(outside_col,inside_col)))
        df_1=df_1.drop(df_1.index[[0,1]]).reset_index(drop=True).drop('с начала месяца', axis=1, level=1).droplevel(level=1, axis=1).rename( columns={'':'region'})
        df_1=df_1.rename(columns={df_1.columns[7]: 'date'})
        df_1=df_1.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    dfs_temp_3.append(df_1)

#time.sleep(3)
dfs_temp_3 = pd.concat(dfs_temp_3).reset_index().drop(columns=['index'])
table_2=dfs_1[1::3]
for t2 in table_2:
    time.sleep(2)
    df_2=pd.DataFrame(t2)
    df_2= df_2.rename(columns=df_2.iloc[0]).drop(df_2.index[0]).reset_index(drop=True).rename( columns={'':'region'})
    df_2=df_2.rename(columns={df_2.columns[8]: 'date'})
    df_2=df_2.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    #df_1 = pd.concat()
    dfs_temp_4.append(df_2)

dfs_temp_4 = pd.concat(dfs_temp_4).reset_index().drop(columns=['index'])

table_3=dfs_1[2::3]
for t3 in table_3:
    time.sleep(2)
    df_3=pd.DataFrame(t3)
    df_3= df_3.rename(columns=df_3.iloc[0]).drop(df_3.index[0]).reset_index(drop=True).rename( columns={'':'region'})
    df_3=df_3.rename(columns={df_3.columns[8]: 'date'})
    df_3=df_3.melt(id_vars=['region', 'date'], var_name="type_of_goods", value_name="price_in_per")
    #df_1 = pd.concat()
    dfs_temp_5.append(df_3)

dfs_temp_5 = pd.concat(dfs_temp_5).reset_index().drop(columns=['index'])
time.sleep(2)


In [ ]:
l_pd_3=[dfs_temp_3, dfs_temp_4, dfs_temp_5]
n_df_1=pd.concat(l_pd_3)

In [13]:
n_df_1

,region,date,type_of_goods,price_in_per
0,По обследованным городам,по состоянию на 15 марта 2022 года,Социально-значимые продовольственные товары,"105,6"
1,Нур-Султан,по состоянию на 15 марта 2022 года,Социально-значимые продовольственные товары,"106,2"
2,Алматы,по состоянию на 15 марта 2022 года,Социально-значимые продовольственные товары,"104,5"
3,Шымкент,по состоянию на 15 марта 2022 года,Социально-значимые продовольственные товары,"105,0"
4,Актау,по состоянию на 15 марта 2022 года,Социально-значимые продовольственные товары,"105,9"
...,...,...,...,...
795,Семей,по состоянию на 9 марта 2022 года,Соль,"100,0"
796,Талдыкорган,по состоянию на 9 марта 2022 года,Соль,"100,0"
797,Тараз,по состоянию на 9 марта 2022 года,Соль,"100,0"
798,Туркестан,по состоянию на 9 марта 2022 года,Соль,"100,0"


In [14]:
dfs_2=[]

In [15]:
def table_read_2(document):
    document = Document("/Users/ajsaule/Desktop/2022/24 mar-17 may/"+file)
    f=re.findall(r'по.*года', document.paragraphs[1].text)
    for i in range(len(document.tables)):
        if i==4:
            table=document.tables[i]
            data = [[cell.text for cell in row.cells] for row in table.rows]
            df = pd.DataFrame(data)
            df['date']=f[0]
            #df.date = df.date.str.replace('по состоянию на ','').copy()
            dfs_2.append(pd.DataFrame(df))

In [16]:
path ='/Users/ajsaule/Desktop/2022/24 mar-17 may/'
files = os.listdir(path)
for file in files:
    time.sleep(2)
    if file.endswith('.docx'):
        table_read_2(file)

In [17]:
dfs_2= pd.concat(dfs_2).reset_index().drop(columns=['index'])
time.sleep(2)
dfs_2= dfs_2.rename(columns=dfs_2.iloc[0]).drop(dfs_2.index[0]).reset_index(drop=True)
time.sleep(2)
dfs_2=dfs_2.rename(columns={dfs_2.columns[21]: 'date', dfs_2.columns[0]: 'type_of_goods'})
time.sleep(2)
dfs_2=dfs_2.melt(id_vars=['type_of_goods', 'date'], var_name="region", value_name="price_in_per")

In [18]:
dfs_2

,type_of_goods,date,region,price_in_per
0,Социально-значимые продовольственные товары,по состоянию на 19 апреля 2022 года,По обследованным городам,"100,6"
1,Мука пшеничная первого сорта,по состоянию на 19 апреля 2022 года,По обследованным городам,"100,7"
2,Хлеб пшеничный из муки первого сорта,по состоянию на 19 апреля 2022 года,По обследованным городам,"99,6"
3,Рожки (весовые),по состоянию на 19 апреля 2022 года,По обследованным городам,"100,9"
4,"Рис шлифованный, полированный (весовой)",по состоянию на 19 апреля 2022 года,По обследованным городам,"100,3"
...,...,...,...,...
3755,Лук репчатый,по состоянию на 5 апреля 2022 года,Туркестан,"106,5"
3756,Капуста белокочанная,по состоянию на 5 апреля 2022 года,Туркестан,"104,2"
3757,Сахар-песок,по состоянию на 5 апреля 2022 года,Туркестан,"101,3"
3758,"Яйца, 1 категории",по состоянию на 5 апреля 2022 года,Туркестан,"101,8"


In [53]:
pdList = [n_df, n_df_1, dfs_2]
final_table= pd.concat(pdList).reset_index().drop(columns=['index'])

In [54]:
final_table

,region,date,type_of_goods,price_in_per
0,Нур-Султан,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,2"
1,Алматы,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,1"
2,Шымкент,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,2"
3,Актау,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,1"
4,Актобе,по 1 марта 2022 года,Социально-значимые продовольственные товары,"100,4"
...,...,...,...,...
8155,Туркестан,по состоянию на 5 апреля 2022 года,Лук репчатый,"106,5"
8156,Туркестан,по состоянию на 5 апреля 2022 года,Капуста белокочанная,"104,2"
8157,Туркестан,по состоянию на 5 апреля 2022 года,Сахар-песок,"101,3"
8158,Туркестан,по состоянию на 5 апреля 2022 года,"Яйца, 1 категории","101,8"


In [56]:
final_table['type_of_goods'] =  final_table['type_of_goods'].apply(lambda x: x.replace('(','').replace(')',''))

In [ ]:
final_table.type_of_goods = final_table.type_of_goods.str.capitalize()

In [57]:
replaces = {'января': '01', 'февраля':'02', 'марта':'03', 'апреля':'04', 'мая':'05', 'июня':'06', 
          'июля':'07','августа':'08', 'сентября':'09', 'октября':'10', 'ноября':'11', 'декабря':'12',
            ' года':'', 'по состоянию на ': '', 'по ':'', '\n':'', '-':'', 'Капустабелокочанная':'Капуста белокочанная',
           '1': ' I', 'категории':'категория', 'Мука пшеничная  первого сорта': 'Мука пшеничная первого сорта', 
           ' весовая':'','шлифованный, полированный весовой':'', ' весовые':'', 'сорта ':'сорта', 'Сахарпесок':'Сахар-песок',
           'cахарпесок':'Сахар-песок','Сахарпесок':'Сахар-песок', 'Рис ':'Рис', 'Астана': 'Нур-Султан', 'НурСултан': 'Нур-Султан'}

In [ ]:
def check_goods(x):
    if x == 'Яйца':
        return 'Яйца, I категории'
    if x.find('Яйца') != -1:
        return 'Яйца, I категории'
    if x=='Мука пшеничная':
        return 'Мука пшеничная из высшего сорта'
    else:
        return x

final_table.type_of_goods = final_table.type_of_goods.apply(lambda x: check_goods(x))


In [58]:
for key, value in replaces.items():
    final_table.date = final_table.date.str.replace(key, value)
    final_table.region = final_table.region.str.replace(key, value)
    final_table.type_of_goods = final_table.type_of_goods.str.replace(key, value)

In [61]:
final_table['date'] =  pd.to_datetime(final_table['date'], format='%d %m %Y')

In [64]:
final_table['type_of_goods'].unique()

array(['Социальнозначимые продовольственные товары',
       'Мука пшеничная первого сорта',
       'Хлеб пшеничный из муки первого сорта', 'Рожки', 'Рис ',
       'Крупа гречневая', 'Масло подсолнечное',
       'Масло сливочное несоленое', 'Говядина лопаточногрудная часть',
       'Мясо кур бедренная и берцовая кость с прилегающей к ней мякотью',
       'Молоко пастеризованное 2,5%', 'Кефир 2,5%', 'Творог', 'Картофель',
       'Морковь', 'Лук репчатый', 'Капуста белокочанная', 'Сахар-песок',
       'Яйца, 1 категория', 'Соль', '\xa0'], dtype=object)

In [65]:
final_table.groupby('type_of_goods').first()

,region,date,price_in_per
type_of_goods,,,
,По обследованным городам,2022-04-12,По обследованным городам
Говядина лопаточногрудная часть,Нур-Султан,2022-03-01,"100,0"
Капуста белокочанная,Нур-Султан,2022-03-01,"110,4"
Картофель,Нур-Султан,2022-03-01,"100,7"
"Кефир 2,5%",Нур-Султан,2022-03-01,"100,0"
Крупа гречневая,Нур-Султан,2022-03-01,"100,0"
Лук репчатый,Нур-Султан,2022-03-01,"103,0"
Масло подсолнечное,Нур-Султан,2022-03-01,"99,0"
Масло сливочное несоленое,Нур-Султан,2022-03-01,"100,0"
